# Task : Implement Attention mechanism without training weights
### Todo:
- Calculate the context vectors for a mock input
- step1: Calculate attention scores
- step2: Normalise the attention scores using softmax
- step3: Calculate context vectors. 
- ***Lets start by taking a mock text input***

In [14]:
import torch 
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [15]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

- Lets calculate attention score for a single input embedding 
-  To do that we will have to find out the dot prodcut between query and every vector in the input

In [16]:
query = inputs[1]

attention_scores1 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attention_scores1[i] = torch.dot(x_i, query)

print(f" Attention score for our query x^2: {attention_scores1}")


 Attention score for our query x^2: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- Now lets normalize these scores with softmax 
- Normalizing helps in esier interpretation of attenion weights, also using it will keep it consistent with other weights during model taining
- Using pytorch implementaion of softmax also helps with navigating overflow issues.

In [17]:
 
attention_weights1 = torch.softmax(attention_scores1, dim=0)
print(f"Attention weights for our query x^2: {attention_weights1}")
print(f"Attention weights sum {attention_weights1.sum()}")

Attention weights for our query x^2: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Attention weights sum 1.0


- Now lets get the context vector for our query. 
- To do that We will have to multiply each attention weight of query x^2 ([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
- with each of the input embedding vecotrs. This will scale the embedding vectors down by the factor of attention weight.

In [18]:
 
context_vec1 = torch.empty(query.shape[0])
for i, x_i in enumerate(inputs):
    context_vec1 += attention_weights1[i] * x_i
print(f"Context vector for our query x^2: {context_vec1}")

Context vector for our query x^2: tensor([0.4419, 0.6515, 0.5683])


- Now that we have understood the steps to calculate the context vector, we can do this for all the input embeddings
- lets first create attention scores for our entire input. Using for loops is slow, so lets use th matrix multiplication.
- all we have to do is multiply inputs with the transpose of inputs
- we will see that we get the same result. 

In [19]:

attention_scores = inputs @ inputs.T
print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


***Notice how the scores for the token 'Journey' are exactly the same as when we calculated them using for loop***
- Now lets normalize the scores using softmax

In [20]:
attention_weights = torch.softmax(attention_scores, dim=-1)
print(attention_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- Now we need to calculate context vectors for each of our input embeddings.
- We could multiply attention_weights(6x6) with the input embeddings(6x3) to get the context vecotrs for each embedding


In [23]:
context_vectors = attention_weights @ inputs 
print(context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
